## Chat with memory

### Install libraries

In [5]:
!pip install -q langchain-openai python-dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


### Load env variables

In [6]:
from dotenv import load_dotenv
load_dotenv()

True

### History
History – stores and injects the entire conversation history message by message into the prompt.

In [7]:
from langchain_core.chat_history import InMemoryChatMessageHistory

# message history
history = InMemoryChatMessageHistory()

history.add_user_message("Buenos dias!")
history.add_ai_message("hello!")
history.add_user_message("Whats your name?")
history.add_ai_message("My name is GIGACHAT")

# history.messages

In [8]:
history.messages

[HumanMessage(content='Buenos dias!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='hello!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Whats your name?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='My name is GIGACHAT', additional_kwargs={}, response_metadata={})]

### Memory

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# message types: user, assistant, system, function, tool
prompt = ChatPromptTemplate.from_messages([
    ("system", "You have a friendly conversation and remember the context."
               "Respond in english."),
    MessagesPlaceholder("history"),
    ("user", "{input}"),
])

chain = prompt | llm

store = {}
def get_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

chain_with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history=get_history,
    input_messages_key="input",
    history_messages_key="history",
)

sid = "demo-session-123"

resp1 = chain_with_memory.invoke(
    {"input": "Hello. My name is Walter White but everyone call me Heisenberg."},
    config={"configurable": {"session_id": sid}}
)
print(resp1.content)

resp2 = chain_with_memory.invoke(
    {"input": "Say my name."},
    config={"configurable": {"session_id": sid}}
)
print(resp2.content)


Hello, Heisenberg! That's quite a memorable name. Are you a fan of the show, or is there another reason you go by that name?
Heisenberg! You’ve definitely made a name for yourself. What’s on your mind today?


### Summary
Summary memory – instead of the full history, passes a condensed summary of previous conversations to the model, which saves tokens and makes it easier to scale long dialogues.

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

summarizer_prompt = ChatPromptTemplate.from_messages([
    ("system", "Summarize the following conversation briefly:"),
    ("human", "{conversation}")
])
summarizer = summarizer_prompt | llm | StrOutputParser()

conversation_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Here is the summary of prior conversation:\n{summary}"),
    MessagesPlaceholder("history"),
    ("human", "{input}")
])
conversation_chain = conversation_prompt | llm | StrOutputParser()

store = {}
summaries = {}

def get_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

def get_summary(session_id: str) -> str:
    if session_id not in summaries:
        summaries[session_id] = "No previous conversation."
    return summaries[session_id]

def update_summary(session_id: str, threshold: int = 4):
    """Update summary and clear old messages when threshold is reached"""
    history = get_history(session_id)

    if len(history.messages) >= threshold:
        current_summary = summaries.get(session_id, "")

        if current_summary and current_summary != "No previous conversation.":
            conversation_text = f"Previous summary: {current_summary}\n\nRecent messages: {history.messages}"
        else:
            conversation_text = str(history.messages)

        new_summary = summarizer.invoke({"conversation": conversation_text})
        summaries[session_id] = new_summary

        history.clear()

chain_with_memory = RunnableWithMessageHistory(
    conversation_chain,
    get_session_history=get_history,
    input_messages_key="input",
    history_messages_key="history"
)

sid = "demo-summary"

cfg = {"configurable": {"session_id": sid}}
summary = get_summary(sid)
response1 = chain_with_memory.invoke({"input": "Hello! My name is Michał.", "summary": summary}, cfg)
print(response1)

# Update summary periodically (e.g., after every 2-3 exchanges)
update_summary(sid, threshold=4)

summary = get_summary(sid)
response2 = chain_with_memory.invoke({"input": "What is my name?", "summary": summary}, cfg)
print(response2)

update_summary(sid, threshold=4)


Hello, Michał! How can I assist you today?
Your name is Michał. How can I help you today?
